In [1]:
from pyspark import SparkConf,SparkContext

conf=SparkConf().setAppName('spark3').setMaster('local[*]')
sc=SparkContext(conf=conf)

22/05/10 18:33:23 WARN Utils: Your hostname, MaQiang resolves to a loopback address: 127.0.1.1; using 192.168.1.110 instead (on interface wlp2s0)
22/05/10 18:33:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/10 18:33:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### countByKey

In [2]:
rdd=sc.textFile('txt/spark3.txt')
rdd1=rdd.flatMap(lambda x: x.split(' ')).map(lambda x: (x,1))

In [3]:
rdd1.countByKey()

defaultdict(int,
            {'hello': 3,
             'txt': 2,
             'test': 3,
             'qiang': 2,
             'hadoop': 1,
             'spark': 2})

### fold

In [4]:
rdd=sc.parallelize(range(1,101),5)

In [5]:
rdd.fold(0,lambda x,y:x+y)  #0作为初始值 + 后面总数

5050

### first , take ,top, count

In [6]:
rdd.first()

1

In [7]:
rdd.take(5)

[1, 2, 3, 4, 5]

In [8]:
rdd.top(5)  #降序

[100, 99, 98, 97, 96]

In [9]:
rdd.count()

100

### takeSample 

随机取出5个

In [10]:
rdd.takeSample(False,5)  #True 允许一样的, False 不同个数据

[10, 83, 96, 56, 84]

### takeOrder

In [11]:
rdd.takeOrdered(5)

[1, 2, 3, 4, 5]

In [12]:
rdd.takeOrdered(5,lambda x:-x) # 倒序

[100, 99, 98, 97, 96]

### foreach 没有返回值 类似map ,效率快

In [13]:
rdd.foreach(lambda x: print(x))  #打印的顺序不对,因为不返回到driver,之在分区处理,更快

21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


### saveAsFile

In [15]:
rdd.saveAsTextFile('txt/file.txt') #分区直接写入

### mapPartitions  
分区算子 更快,map每次传递一个数据,mappartitions 返回一个分区, 返回 RDD

In [17]:
def fun(x):
    result=[]
    for i in x:
        result.append(i*10)
    return result

In [19]:
rdd.mapPartitions(fun).collect()

[10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 210,
 220,
 230,
 240,
 250,
 260,
 270,
 280,
 290,
 300,
 310,
 320,
 330,
 340,
 350,
 360,
 370,
 380,
 390,
 400,
 410,
 420,
 430,
 440,
 450,
 460,
 470,
 480,
 490,
 500,
 510,
 520,
 530,
 540,
 550,
 560,
 570,
 580,
 590,
 600,
 610,
 620,
 630,
 640,
 650,
 660,
 670,
 680,
 690,
 700,
 710,
 720,
 730,
 740,
 750,
 760,
 770,
 780,
 790,
 800,
 810,
 820,
 830,
 840,
 850,
 860,
 870,
 880,
 890,
 900,
 910,
 920,
 930,
 940,
 950,
 960,
 970,
 980,
 990,
 1000]

### foreachPartition 
相当于foreach,但是直接操作分区

In [21]:
def fun(x):
    result=[]
    for i in x:
        result.append(i*10)
    print( result)
rdd.foreachPartition(fun)

[410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600]
[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]
[210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400]
[610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800]
[810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990, 1000]


### partitionBy

自定义规则 分区

In [28]:
rdd=sc.textFile('txt/spark3.txt').flatMap(lambda x:x.split(' ')).map(lambda x :(x,1))

In [33]:
def process(x):
    if 'spark'==x : return 0  #给第一分区
    if 'hello'==x : return 1
    return 2

In [35]:
rdd.partitionBy(3,process).glom().collect()

[[('spark', 1), ('spark', 1)],
 [('hello', 1), ('hello', 1), ('hello', 1)],
 [('txt', 1),
  ('txt', 1),
  ('test', 1),
  ('test', 1),
  ('test', 1),
  ('qiang', 1),
  ('qiang', 1),
  ('hadoop', 1)]]

### repartition 最好不用,会洗牌,影响速度
重新分区

In [38]:
rdd.repartition(3).glom().collect()

[[],
 [('hello', 1),
  ('hello', 1),
  ('txt', 1),
  ('txt', 1),
  ('test', 1),
  ('test', 1),
  ('test', 1),
  ('qiang', 1),
  ('qiang', 1)],
 [('hello', 1), ('hadoop', 1), ('spark', 1), ('spark', 1)]]

22/05/10 22:04:10 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 8807612 ms exceeds timeout 120000 ms
22/05/10 22:04:10 WARN SparkContext: Killing executors is not supported by current scheduler.
